In [16]:
import os

path = "Dataset/training_dataset/training_data/images"

# files = [f for f in glob.glob(path + "/.mat", recursive=True)]
file = os.listdir(path)
file[:10]

['Poselet_318.jpg',
 'VOC2010_724.jpg',
 'Inria_429.jpg',
 'VOC2010_730.jpg',
 'Inria_367.jpg',
 'Poselet_324.jpg',
 'VOC2010_718.jpg',
 'Skin_194.jpg',
 'Inria_401.jpg',
 'Inria_415.jpg']

In [2]:
# for item in file:
#     item = item.replace(".jpg","")
#     print(item)

In [2]:
dataset = {}
dataset['info'] = {} 
dataset['info']["contributor"] = "Vin-AI Lab"
dataset['info']["date_created"] = '4/5/2020' 
dataset['info']["description"] = "Hand-Dataset" 
dataset['info']["url"] = "https://github.com/oggyfaker"
dataset['info']["Version"] = "1.0"
dataset['info']["year"] = "2020"

In [3]:
license = {} 
license['id'] = 1 
license['name'] = "VinAi Lab"
license['url'] = "https://github.com/oggyfaker"

dataset["licenses"] = []
dataset["licenses"].append(license)
print(dataset["licenses"])

[{'id': 1, 'name': 'VinAi Lab', 'url': 'https://github.com/oggyfaker'}]


In [4]:
# Create dictionary of name and height and weight
import cv2
image_dict = {}
for image_item in file:
    if ".jpg" in image_item:
        im = cv2.imread(path +"/" +image_item)
        h, w, c = im.shape
        image_dict[image_item] = [w,h]

In [6]:
# Images
dataset['image'] = []
for i,image_name in enumerate(image_dict):
    dict_tmp = {}
    dict_tmp['file_name'] = image_name
    dict_tmp['width'] ,dict_tmp['height'] = image_dict[image_name]
    dict_tmp["id"] = i 
    dict_tmp['license'] = 1
    dict_tmp["flickr_url"] = ""
    dict_tmp["coco_url"] = ""
    dict_tmp["date_captured"] = "4/5/2020"
    dataset['image'].append(dict_tmp)


In [7]:
# Category 
dict_temp_cat = {}
dict_temp_cat['id'] = 1 
dict_temp_cat['name'] = "Hand"
dict_temp_cat["supercategory"] = "Body"

dataset['categories'] = []
dataset['categories'].append(dict_temp_cat)

In [8]:
# !pip install mat4py

In [9]:
from mat4py import loadmat
#-----------------------------------------------
# xmin = d2 ; xmax = b2 | ymin = a1 , ymax = c1 |
#-----------------------------------------------
# Error = []
dataset['annotation'] = []
count_id = 0
for item_image in file:
#     flag_saved = False
    if ".jpg" in item_image:
        file_math = item_image.replace(".jpg",".mat")
        data = loadmat('Dataset/training_dataset/training_data/annotations/'+ file_math)
    
        # Có trường hợp bbox chỉ có 1 object == dict : nên cần chuyển sang list chứa dict đó 
        if type(data['boxes']) == dict:
            data['boxes']= [data['boxes']]
        
        #Check number object 
        if len(data['boxes']) == 1: 
            iscrowd = 0
        elif len(data['boxes']) > 1:
            iscrowd = 1
        else:
            print("ko có annotation")
            break 
    
        # Get id image 
        for image in dataset['image']:
            if image['file_name'] == item_image:
                id_image = image['id']
                break 
        # Get box
        for box in data['boxes']:
            xmin = box['d'][1]
            xmax = box['b'][1]
            ymin = box['a'][0]
            ymax = box['c'][0]
            # Change to CoCo 
            width = xmax - xmin 
            height = ymax - ymin 
            
            dict_anotate_temp = {}
            dict_anotate_temp['id'] = count_id
            count_id += 1 
            dict_anotate_temp["category_id"] = 1 
            dict_anotate_temp["image_id"] = id_image
            dict_anotate_temp["iscrowd"] = iscrowd
            dict_anotate_temp["segmentation"] = [[]]
            dict_anotate_temp["area"] = width *  height
            dict_anotate_temp["bbox"] = [xmin,ymin,width,height]
        
            dataset['annotation'].append(dict_anotate_temp)
print("Done!!! ")
    

Done!!! 


In [12]:
# dataset

In [14]:
ls

Baseline.ipynb    Dataset/          Result Json/      Validation.ipynb
Check.ipynb       Result Image/     Train.ipynb


In [15]:
import json
with open("Result Json/detection_train2020.json", "w") as outfile: 
    json.dump(dataset, outfile) 